In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 


In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.shape

(404290, 6)

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
new_df = df.sample(30000)

In [6]:
new_df.duplicated().sum()

0

In [8]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
178706,How will World War 3 be like?,How do you think a World War III would play out?
298236,Is it true that all education in Norway is free?,Is it true that Taiwan university provide free...
48279,What Is your New year resolutions in 2017?,What is your resolution for 2017?
309324,Why do I want to start smoking?,I don't want to start smoking. What should I do?
43811,Which is the best site to download oracle ebooks?,How to download FIFA 17 for PC?


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
# merge text 
question = list(ques_df['question1'])+list(ques_df['question2'])
cv= CountVectorizer(max_features=3000)
q1_arr,q2_arr = np.vsplit(cv.fit_transform(question).toarray(),2)

In [10]:
temp_df1 = pd.DataFrame(q1_arr,index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr,index=ques_df.index)
temp_df= pd.concat([temp_df1,temp_df2],axis=1)
temp_df.shape

(30000, 6000)

In [11]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [12]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
178706,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
298236,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48279,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
309324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43811,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=42)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7408333333333333

In [16]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score( y_test, y_pred)

0.7306666666666667